ДЗ - построение ассоциативных правил для датасета :
Можно использовать например apriori, association_rules, fpgrowth из mlxtend.frequent_patterns (работают чуть быстрее).
Написать выводы. 


https://www.kaggle.com/datasets/gorkhachatryan01/purchase-behaviour?select=dataset.csv

In [37]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import re
import csv
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth, association_rules
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx



,pork,sandwich bags,lunch meat,all- purpose,flour,soda,butter,vegetables,beef,aluminum foil,all- purpose.1,dinner rolls,shampoo,all- purpose.2
0,shampoo,hand soap,waffles,vegetables,cheeses,mixes,milk,sandwich bags,laundry detergent,dishwashing liquid/detergent,waffles,individual meals,hand soap,vegetables
1,pork,soap,ice cream,toilet paper,dinner rolls,hand soap,spaghetti sauce,milk,ketchup,sandwich loaves,poultry,toilet paper,ice cream,ketchup
2,juice,lunch meat,soda,toilet paper,all- purpose,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,pasta,tortillas,mixes,hand soap,toilet paper,vegetables,vegetables,paper towels,vegetables,flour,vegetables,pork,poultry,eggs
4,toilet paper,eggs,toilet paper,vegetables,bagels,dishwashing liquid/detergent,cereals,paper towels,laundry detergent,butter,cereals,bagels,paper towels,shampoo


In [2]:


# Чтение CSV-файла в виде списка списков
data = []
with open('dataset.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        data.append(row)

In [13]:
data[1498]

[' laundry detergent',
 ' vegetables',
 ' shampoo',
 ' vegetables',
 ' ',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [6]:
# Очистим данные от пустых значений '',  ' '
basket = pd.Series(data)
basket = basket.apply(lambda x: [item for item in x if item != ''])
basket = basket.apply(lambda x: [item for item in x if item != ' '])

In [11]:
basket.shape


(1499,)

In [7]:
basket

0       [ pork,  sandwich bags,  lunch meat,  all- pur...
1       [ shampoo,  hand soap,  waffles,  vegetables, ...
2       [ pork,  soap,  ice cream,  toilet paper,  din...
3       [ juice,  lunch meat,  soda,  toilet paper,  a...
4       [ pasta,  tortillas,  mixes,  hand soap,  toil...
                              ...                        
1494    [ beef,  sandwich bags,  hand soap,  paper tow...
1495    [ dinner rolls,  lunch meat,  spaghetti sauce,...
1496    [ lunch meat,  eggs,  poultry,  vegetables,  t...
1497    [ ketchup,  milk,  poultry,  cheeses,  soap,  ...
1498    [ laundry detergent,  vegetables,  shampoo,  v...
Length: 1499, dtype: object

In [8]:
basket[1498]

[' laundry detergent', ' vegetables', ' shampoo', ' vegetables']

In [14]:
encoder = TransactionEncoder()
transactions = pd.DataFrame(encoder.fit(basket).transform(basket), columns=encoder.columns_)
display(transactions.head())

,all- purpose,aluminum foil,bagels,beef,butter,cereals,cheeses,coffee/tea,dinner rolls,dishwashing liquid/detergent,...,shampoo,soap,soda,spaghetti sauce,sugar,toilet paper,tortillas,vegetables,waffles,yogurt
0,True,True,False,True,True,False,False,False,True,False,...,True,False,True,False,False,False,False,True,False,False
1,False,False,False,False,False,False,True,False,False,True,...,True,False,False,False,False,False,False,True,True,False
2,False,False,False,False,False,False,False,False,True,False,...,False,True,False,True,False,True,False,False,False,False
3,True,False,False,False,False,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,True,True,False,False


Алгоритм Apriori будет использоваться для генерации частых наборов элементов. Мы зададим минимальную поддержку в размере 6 из общего числа транзакций. Генерируются правила ассоциаций, и мы отфильтровываем значения Lift > 1,3.

In [49]:
frequent_itemsets = apriori(transactions, min_support= 6/len(basket), use_colnames=True, max_len = 4)
rules = association_rules(frequent_itemsets, metric="lift",  min_threshold = 1.45)
display(rules.head(10))
print("Rules identified: ", len(rules))

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"( eggs, all- purpose)",( bagels),0.071381,0.278185,0.029353,0.411215,1.478204,0.009496,1.225939,0.348371
1,( bagels),"( eggs, all- purpose)",0.278185,0.071381,0.029353,0.105516,1.478204,0.009496,1.038161,0.448181
2,"( beef, all- purpose)",( fruits),0.068045,0.263509,0.026017,0.382353,1.451005,0.008087,1.192414,0.333517
3,( fruits),"( beef, all- purpose)",0.263509,0.068045,0.026017,0.098734,1.451005,0.008087,1.034051,0.422032
4,"( aluminum foil, cereals)",( mixes),0.069380,0.273516,0.028019,0.403846,1.476501,0.009042,1.218619,0.346783
5,( mixes),"( aluminum foil, cereals)",0.273516,0.069380,0.028019,0.102439,1.476501,0.009042,1.036833,0.444226
6,"( aluminum foil, tortillas)",( sandwich loaves),0.058706,0.248833,0.022015,0.375000,1.507038,0.007407,1.201868,0.357430
7,( sandwich loaves),"( aluminum foil, tortillas)",0.248833,0.058706,0.022015,0.088472,1.507038,0.007407,1.032655,0.447898
8,"( dinner rolls, bagels)",( tortillas),0.067378,0.244163,0.024016,0.356436,1.459828,0.007565,1.174455,0.337744
9,( tortillas),"( dinner rolls, bagels)",0.244163,0.067378,0.024016,0.098361,1.459828,0.007565,1.034362,0.416740


Rules identified:  53514


Вывод: мы получение значение lift больше 1. Это говорит о том что существует связь между продуктами. Конечно то, что значение не превышает  1,5 говорит что связь не большая. 